In [2]:
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing
import copy
from torch_geometric.utils import degree

import torch
from torch import nn, optim, matmul

from torch_sparse import SparseTensor, matmul

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj

from scipy import sparse

In [3]:
'''
Let's look at the following 3 x 4 interaction matrix
                items
                i1  i2  i3  i4
    users   u1  0   0   1   1
            u2  1   0   1   0
            u3  0   1   0   0   

'''

"\nLet's look at the following 3 x 4 interaction matrix\n                items\n                i1  i2  i3  i4\n    users   u1  0   0   1   1\n            u2  1   0   1   0\n            u3  0   1   0   0   \n\n"

In [4]:
# Making an interaction matrix
r_mat = np.array([
    [0, 0, 1, 1], 
    [1, 0, 1, 0], 
    [0, 1, 0, 0]
])

print(r_mat)

[[0 0 1 1]
 [1 0 1 0]
 [0 1 0 0]]


In [5]:
# Converting the interaction matrix into the coo format
r_mat_np_coo = sparse.coo_matrix(r_mat) # This is more space efficent

print(r_mat_np_coo)

  (0, 2)	1
  (0, 3)	1
  (1, 0)	1
  (1, 2)	1
  (2, 1)	1


In [7]:
# edge index
r_mat_edge_index = [r_mat_np_coo.row, r_mat_np_coo.col]

print(r_mat_edge_index[0])
print(r_mat_edge_index[1])

[0 0 1 1 2]
[2 3 0 2 1]


## We can do the same thing in pytorch

In [9]:
r_mat_edge_index = torch.LongTensor(np.array(r_mat_edge_index))

r_mat_edge_index

tensor([[0, 0, 1, 1, 2],
        [2, 3, 0, 2, 1]])

In [10]:
sparse_r_mat_edge_index = SparseTensor(
    row=r_mat_edge_index[0], 
    col=r_mat_edge_index[1], 
    sparse_sizes=(3, 4)
)

print(sparse_r_mat_edge_index)

SparseTensor(row=tensor([0, 0, 1, 1, 2]),
             col=tensor([2, 3, 0, 2, 1]),
             size=(3, 4), nnz=5, density=41.67%)


In [11]:
print(sparse_r_mat_edge_index.to_dense())

tensor([[0., 0., 1., 1.],
        [1., 0., 1., 0.],
        [0., 1., 0., 0.]])


In [12]:
# Creating a function to convert interaction matrix edge index to adjecency matrix edge index

def convert_r_mat_edge_index_to_adj_mat_index(input_edge_index, row_size, col_size):
    R = torch.zeros((row_size, col_size))

    # Convert sparse Coo format to dense format to get R
    for i in range(len(input_edge_index[0])):
        row_idx = input_edge_index[0][i]
        col_idx = input_edge_index[0][i]
        R[row_idx][col_idx] = 1
    
    # Perform the r_mat to adj_mat converstion
    R_transpose = torch.transpose(R, 0, 1)
    adj_mat = torch.zeros((row_size + col_size, row_idx + col_size))

    adj_mat[:, row_size, row_size: ] = R.clone()
    adj_mat[row_size: , : row_size] = R_transpose.clone()

    # Convertfrom dense format back to sparse format so we can get the edge_index of the adjecency matrix
    adj_mat_coo = adj_mat.to_sparse_coo()
    adj_mat_coo = adj_mat_coo.indices()
    return adj_mat_coo

In [ ]:
def convert_adj_mat_edge_index_to_r_mat_edge_index(input_edge_index, row_size, col_size):
    # Create a sparse tensor so we can easlity do the to_dense conversation and get sub amtrix 
    sparse_input_edge_index = SparseTensor(
        row=input_edge_index[0], 
        col=input_edge_index[1], 
        sparse_sizes=((row_size + col_size), row_size + col_size)
    )

    adj_mat = sparse_r_mat_edge_index.to_dense()
    interact_matrix = adj_mat[:, row_size, col_size : ]

    r_mat_edge_index = interact_matrix.to_sparse_coo().indices()

    return r_mat_edge_index